In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
#path's csv file, 
nov = pd.read_csv('/Users/tommy/Desktop/codici/archive/2019-Nov.csv', header='infer',
parse_dates=['event_time'],
date_parser=pd.to_datetime)

In [14]:
#PURCHASE RATE: => #NUMBER OF PRODUCT SOLD/#USERS
subRQ6 = nov[['event_type','user_id','category_code','product_id']]


In [19]:
# obtaining the number of product sold:
# by taking only rows about purchasing
# and at the end we count them (.shape[0])
# shape[0] are the rows of the dataset where the event is equal to purchase
# so it rapresents the product sold

prod_sold = subRQ6[subRQ6['event_type']=='purchase'].shape[0]

In [17]:
# in a similar way we calculate the number of user:
# this time we group by 'user_id' so we obtain the exact users's number
# we extract that number by the command shape[0] (number of rows)
users = subRQ6.groupby('user_id').product_id.count().shape[0]

In [23]:
#finally we calculate the actual purchase rate value
purchase_rate = prod_sold/users
round(purchase_rate,4)

0.2481

In [33]:
type(prod_sold)

int

In [ ]:
#Now that we have the purchase rate we can calculate the conversion rate
# CONVERSION RATE = PURCHASE RATE/ #VIEWS

In [37]:
#same think of before
views =     subRQ6[subRQ6['event_type']=='view'].shape[0]

In [39]:
#conversion rate calculated by following the previous formula
conversion_rate = purchase_rate / views
conversion_rate

3.903349387514857e-09

In [47]:
subRQ6 = subRQ6.dropna()

In [48]:
#purchase rate of each category
subRQ6['category_code'] = [el.split('.')[0] for el in list(subRQ6['category_code']) if type(el) == str]
#number_of_sales = subRQ6.groupby('category_code').product_id.count()

<ipython-input-48-434ce4d17c6e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subRQ6['category_code'] = [el.split('.')[0] for el in list(subRQ6['category_code']) if type(el) == str]


In [51]:
number_of_sales = subRQ6.groupby('category_code').product_id.count()

In [52]:
number_of_sales

category_code
accessories       390038
apparel          3011101
appliances       8489825
auto             1214159
computers        4181358
construction     1092155
country_yard       17719
electronics     24006086
furniture        2111829
kids              806384
medicine           22382
sport             247994
stationery         12778
Name: product_id, dtype: int64

In [54]:
user_of_each_cat = subRQ6.groupby('category_code').user_id.count()

In [55]:
user_of_each_cat

category_code
accessories       390038
apparel          3011101
appliances       8489825
auto             1214159
computers        4181358
construction     1092155
country_yard       17719
electronics     24006086
furniture        2111829
kids              806384
medicine           22382
sport             247994
stationery         12778
Name: user_id, dtype: int64

In [56]:
purchase_rate_for_cat = number_of_sales.divide(user_of_each_cat)
purchase_rate_for_cat

category_code
accessories     1.0
apparel         1.0
appliances      1.0
auto            1.0
computers       1.0
construction    1.0
country_yard    1.0
electronics     1.0
furniture       1.0
kids            1.0
medicine        1.0
sport           1.0
stationery      1.0
dtype: float64